In [1]:
# !pip install -U spacy -q

In [1]:
import spacy

In [2]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("id") # load a new spacy model
db = DocBin() # create a DocBin object

In [3]:
import json
f = open('responsibility.json')
train_data = json.load(f)

In [4]:
f = open('qualification.json')
val_data = json.load(f)

In [7]:
for text, annot in tqdm(train_data['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 113/113 [00:00<00:00, 3777.18it/s]


In [8]:
for text, annot in tqdm(val_data['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./validation_data.spacy") # save the docbin object

100%|██████████| 88/88 [00:00<00:00, 4624.60it/s]


In [9]:
! python -m spacy init config config.cfg --lang id --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only), install the
spacy-transformers package and re-run this command. The config generated now
does not use transformers.
ℹ Generated config template specific for your use case
- Language: id
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-05-09 06:10:57.629188: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-09 06:10:57.630261: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./validation_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.43    0.00    0.00    0.00    0.00
 14     200         52.27   1516.21   95.94   96.17   95.71    0.96
 33     400        106.68    129.52   94.76   90.83   99.05    0.95
 55     600          0.01      0.01   96.93   96.24   97.62    0.97
 83     800          0.00      0.00   97.16   96.70   97.62    0.97
117    1000          0.00      0.00   98.55  100.00   97.14    0.99
158    1200          0.00      0.00   97.84   98.55   97.14    0.98
208    1400          0.00      0.00   97.40   96.71   98.10    0.97
270    1600          0.00      0.00 

2022-05-09 06:11:17.640750: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-09 06:11:17.641229: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-05-09 06:11:24,372] [INFO] Set up nlp object from config
[2022-05-09 06:11:24,381] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-09 06:11:24,384] [INFO] Created vocabulary
[2022-05-09 06:11:24,385] [INFO] Finished initializing nlp object
[2022-05-09 06:11:24,533] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [11]:
ner_model = spacy.load('model-best')

In [12]:
doc = ner_model('''
"intent": "qualification-DS",
"apa kompetensi yang dibutuhkan role data scientist?",
"apa kompetensi yang dibutuhkan posisi data analis?",
"kompetensi buat enginer apa aja?",
"data science harus bisa apa?",
"lulusan non teknik bisa daftar jadi scientist?",
"kalau ga bisa IT bisa jadi analis?",
"kalau belum punya pengalaman bisa jadi engineer?",
"kualifikasi data analyst gimana?",
"daftar jadi data analitik harus bisa apa aja?",
"kalau jurusan tidak tertulis apa bisa jadi data scientis?",
"bagaimana kualifikasi data engine yang dicari?",
"kalau ga bisa data science boleh daftar data scientist?",
"ga bisa coding boleh daftar data science?",
"ga bisa coding boleh daftar data scientist?",
"ga ngerti data science tapi mau daftar",
"belum punya pengalaman mau daftar data scientist",
"pengalaman lebih dari tiga tahun mau lamar data scientist"
"Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"
''')

In [13]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [17]:
doc = ner_model('''
data scientist itu apa, data engineer, analis, data analyst, data analist, data science, DS, DA, DE, science, analyst, engine, engineer
''')

In [18]:
doc.ents

(data scientist,
 data engineer,
 analis,
 data analyst,
 data analist,
 data science,
 DS,
 DA,
 DE,
 analyst,
 engineer)

In [19]:
for ent in doc.ents:
  print(ent.text, ent.label_)

data scientist SCIENTIST
data engineer ENGINEER
analis ANALIS
data analyst ANALIS
data analist ANALIS
data science SCIENTIST
DS SCIENTIST
DA ANALIS
DE ENGINEER
analyst ANALIS
engineer ENGINEER


In [20]:
for ent in doc.ents:
  print(ent.label_)

SCIENTIST
ENGINEER
ANALIS
ANALIS
ANALIS
SCIENTIST
SCIENTIST
ANALIS
ENGINEER
ANALIS
ENGINEER


In [21]:
cek = ner_model('''
Data scientis ngapain aja
''')

In [22]:
for ent in cek.ents:
    print(ent.label_)

SCIENTIST


In [23]:
for ent in cek.ents:
    print(ent.label_)
    if ent.label_ == "ANALIS":
        print('yes')

SCIENTIST
